# Head Neck Cancer Medication Data Mining

Preprocess and predict from the data early and late stage cancer patients

In [71]:
import pandas as pd

df = pd.read_csv('./data/Case_Studies/CaseStudy14_HeadNeck_Cancer_Medication.csv');
df.head()

,PID,ENC_ID,seer_stage,MEDICATION_DESC,MEDICATION_SUMMARY,DOSE,UNIT,FREQUENCY,TOTAL_DOSE_COUNT
0,10000,46836,1,ranitidine,(Zantac) 150 mg tablet oral two times a day,150,mg,two times a day,5
1,10008,46886,1,heparin injection,"5,000 unit subcutaneous three times a day",5000,unit,three times a day,3
2,10029,47034,4,ampicillin/sulbactam IVPB UH,(Unasyn) 15 g IV every 6 hours,1.5,g,every 6 hours,11
3,10063,47240,1,fentaNYL injection UH,25 - 50 microgram IV every 5 minutes PRN sever...,50,microgram,every 5 minutes,2
4,10071,47276,9,simvastatin,(Zocor) 40 mg tablet oral at bedtime,40,mg,at bedtime,1


In [72]:
import numpy as np
import string
import re

hn_corpus = df['MEDICATION_SUMMARY'];
# hn_corpus
hn_corpus = hn_corpus.str.lower();
hn_corpus = hn_corpus.apply(lambda elem: elem.translate(str.maketrans('', '', string.punctuation)));
hn_corpus = hn_corpus.apply(lambda elem: re.sub(r"\n|\r", ' ', elem));
hn_corpus = hn_corpus.apply(lambda elem: elem.translate(str.maketrans('', '', string.digits)));
hn_corpus = hn_corpus.apply(lambda elem: re.sub(r' +', ' ', elem));
hn_corpus = hn_corpus.apply(lambda elem: elem + ' ');

df['MEDICATION_SUMMARY'] = hn_corpus;

# ''.join(hn_corpus)
df['MEDICATION_SUMMARY'].head()

0               zantac mg tablet oral two times a day 
1                 unit subcutaneous three times a day 
2                             unasyn g iv every hours 
3     microgram iv every minutes prn severe pain ma...
4                     zocor mg tablet oral at bedtime 
Name: MEDICATION_SUMMARY, dtype: object

In [73]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 

stopwords = set(STOPWORDS).union(['mg', 'iv']);
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='black',
                stopwords = stopwords,
                collocations=False,
                max_words=200).generate(''.join(df[df["seer_stage"] <= 3]['MEDICATION_SUMMARY']));

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None);
plt.imshow(wordcloud);
plt.axis("off");
plt.tight_layout(pad = 0);

plt.show();
# wordcloud.words_

In [74]:
df['seer_stage'] = df['seer_stage'].apply(lambda elem: int(elem <= 3))

# df['seer_stage'].head()

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer();
dtm = vectorizer.fit_transform(df['MEDICATION_SUMMARY']);

dtm_features = np.array(vectorizer.get_feature_names())
# dtm.getnnz(axis=1)
print(str(Ytrain.shape) + ' - ' + str(dtm.shape) + ' - ' + str(dtm_features.shape))

(529,) - (662, 450) - (450,)


In [76]:
dtm_row_count = dtm.getnnz(axis=0);

dtm = dtm[:, dtm_row_count > 5];
dtm_features = dtm_features[dtm_row_count > 5];
dtm.shape

(662, 137)

In [77]:
from wordcloud import STOPWORDS

is_stopword = np.array([item not in STOPWORDS for item in dtm_features]);
dtm_features = dtm_features[is_stopword];
dtm = dtm[:, is_stopword];
dtm.shape

(662, 115)

In [78]:
from sklearn.model_selection import train_test_split

X = dtm.toarray();
Y = df['seer_stage']

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2);

In [79]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB(var_smoothing=2);
y_pred = model_nb.fit(Xtrain, Ytrain);

train_score = model_nb.score(Xtrain, Ytrain);
test_score = model_nb.score(Xtest, Ytest);

# print(str(Ytrain.shape) + ' - ' + str(dtm_train.shape))
print(train_score);
print(test_score);

0.6616257088846881
0.5939849624060151


In [80]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model_nb, X, Y, cv=5);
print(np.mean(scores));

0.6480405559352928
